In [1]:
from newspaper import Article
import newspaper
from newspaper import news_pool
import pandas as pd
import re
from datetime import datetime
import os
import sys

In [2]:
#Setting default working directory
os.chdir('C:/Users/Hetal/Desktop/SJSU/CMPE - 255/Project/bbc')

In [3]:
#list of news papers to crawl and the categories to look for

papers_dict = {   
    'BBC': {
        'URL':'http://www.bbc.com',
    },
    'BBC_UK': {
        'URL':'http://www.bbc.co.uk',
    },
    'SLATE': {
        'URL':'http://www.slate.com',
    },
    'GUARDIAN': {
        'URL':'http://www.theguardian.com',
    },
    'BREITBRAT': {
        'URL':'http://www.breitbart.com',
    },
    'NBC': {
        'URL':'http://www.nbcnews.com',
    },
    'WASHINGTONPOST': {
        'URL':'http://www.washingtonpost.com',
    },
    'USNEWS': {
        'URL':'http://www.usnews.com/', 
    },
    'NYTIMES': {
        'URL':'http://www.nytimes.com/', 
    },
    'USATODAY': {
        'URL':'http://www.usatoday.com/', 
    },
    'NYDAILYNEWS': {
        'URL':'http://www.nydailynews.com', 
    },
    'LATIMES': {
        'URL':'http://www.latimes.com/', 
    },
    'WALLSTREETJOURNAL': {
        'URL':'http://www.wsj.com/', 
    },
    'BUSINESS_STANDARD': {
        'URL':'http://www.business-standard.com/', 
    },
    'LIVEMINT': {
        'URL':'http://www.livemint.com', 
    }
}

In [4]:
def downloadContent(paper):

    patterns = ['/politics/','/tech/','/technology/','/business/','/entertainment/','/health/']
    type = []
    news = []
    url =  []
    count = 0
    for article in paper.articles:
        try:
            for pattern in patterns:
                if(re.search(pattern,article.url)):
                    if article.url not in url:
                        article.download()
                        article.parse()
                        if(len(article.text) > 350 and len(article.text)<6000):
                            news.append(article.text)
                            type.append(pattern[1:-1])
                            count = count + 1
        except Exception:
            pass

    print("No of articles crawled : ", count);
    combined_data = {'category': type, 'news': news}
    dataset = pd.DataFrame(combined_data)
    return dataset

In [5]:
result = pd.DataFrame()
for name,value in papers_dict.items():
    print("Crawling from: ", value['URL'])
    paper = newspaper.build(value['URL'], language = 'en')
    df = downloadContent(paper)
    result = pd.concat([df, result], ignore_index=True, sort = False)

Crawling from:  http://www.bbc.com
No of articles crawled :  4
Crawling from:  http://www.bbc.co.uk
No of articles crawled :  0
Crawling from:  http://www.slate.com
No of articles crawled :  9
Crawling from:  http://www.theguardian.com
No of articles crawled :  6
Crawling from:  http://www.breitbart.com
No of articles crawled :  258
Crawling from:  http://www.nbcnews.com
No of articles crawled :  0
Crawling from:  http://www.washingtonpost.com
No of articles crawled :  16
Crawling from:  http://www.usnews.com/
No of articles crawled :  10
Crawling from:  http://www.nytimes.com/
No of articles crawled :  28
Crawling from:  http://www.usatoday.com/
No of articles crawled :  48
Crawling from:  http://www.nydailynews.com
No of articles crawled :  23
Crawling from:  http://www.latimes.com/
No of articles crawled :  12
Crawling from:  http://www.wsj.com/
No of articles crawled :  1
Crawling from:  http://www.business-standard.com/
No of articles crawled :  42
Crawling from:  http://www.livem

In [7]:
# dropping ALL duplicte values 
result.drop_duplicates(subset ="news", 
                     keep = "first", inplace = True)
result.loc[(result.category == 'tech'),'category']='technology'
result.loc[(result.category == 'sport'),'category']='sports'
result['category'].value_counts()

politics         91
sports           66
technology       29
entertainment    28
world            20
business         16
Name: category, dtype: int64

In [9]:
with open('../dataset.csv', 'a+', encoding="utf-8-sig", newline='') as f:
    result.to_csv(f, header=False, index = False)